# MTL

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')


df = pd.read_csv(r'.csv')
output_names = ['Electricity Consumption', 'Humidity Closer to Evaporator (%)', 'Temperature Closer to Evaporator (C)']

from MTL_FDCS_1_pre_processing import preprocess_data
X_train_normalized, y_train_normalized, X_val_normalized, y_val_normalized, X_test_normalized, y_test_normalized = preprocess_data(df)

# Combine the training and validation sets directly after they have been recalled
X_train_val_normalized = np.concatenate([X_train_normalized, X_val_normalized], axis=0)
y_train_val_normalized = np.concatenate([y_train_normalized, y_val_normalized], axis=0)

print("Shape of X_train_normalized:", X_train_normalized.shape)
print("Shape of y_train_normalized:", y_train_normalized.shape)
print("Shape of X_val_normalized:", X_val_normalized.shape)
print("Shape of y_val_normalized:", y_val_normalized.shape)
print("Shape of X_test_normalized:", X_test_normalized.shape)
print("Shape of y_test_normalized:", y_test_normalized.shape)
print("Shape of X_train_val_normalized:", X_train_val_normalized.shape)
print("Shape of y_train_val_normalized:", y_train_val_normalized.shape)

Shape of X_train_normalized: (1272, 85)
Shape of y_train_normalized: (1272, 3)
Shape of X_val_normalized: (336, 85)
Shape of y_val_normalized: (336, 3)
Shape of X_test_normalized: (336, 85)
Shape of y_test_normalized: (336, 3)
Shape of X_train_val_normalized: (1608, 85)
Shape of y_train_val_normalized: (1608, 3)


In [2]:
from FDCS_1_MLT_FS import evaluate_model
final_df = evaluate_model(X_train_val_normalized, y_train_val_normalized, X_test_normalized, y_test_normalized, output_names)
final_df

2024-03-07 15:14:49.337740: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-07 15:14:49.340780: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-07 15:14:49.382760: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-07 15:14:49.383543: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-07 15:14:50.124065: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

Epoch 1/100
9/9 [==============================] - 1s 7ms/step - loss: 0.2818 - dense_2_loss: 0.0913 - dense_3_loss: 0.0567 - dense_4_loss: 0.1338 - dense_2_mse: 0.0913 - dense_3_mse: 0.0567 - dense_4_mse: 0.1338
Epoch 2/100
9/9 [==============================] - 0s 3ms/step - loss: 0.1738 - dense_2_loss: 0.0503 - dense_3_loss: 0.0199 - dense_4_loss: 0.1036 - dense_2_mse: 0.0503 - dense_3_mse: 0.0199 - dense_4_mse: 0.1036
Epoch 3/100
9/9 [==============================] - 0s 4ms/step - loss: 0.1264 - dense_2_loss: 0.0456 - dense_3_loss: 0.0204 - dense_4_loss: 0.0604 - dense_2_mse: 0.0456 - dense_3_mse: 0.0204 - dense_4_mse: 0.0604
Epoch 4/100
9/9 [==============================] - 0s 3ms/step - loss: 0.0959 - dense_2_loss: 0.0390 - dense_3_loss: 0.0184 - dense_4_loss: 0.0384 - dense_2_mse: 0.0390 - dense_3_mse: 0.0184 - dense_4_mse: 0.0384
Epoch 5/100
9/9 [==============================] - 0s 3ms/step - loss: 0.0848 - dense_2_loss: 0.0340 - dense_3_loss: 0.0157 - dense_4_loss: 0.0352 -

,True,Pred,Model,Output,Feature_Selection
0,0.018988,0.023318,MTL,Electricity Consumption,Without
1,0.019199,-0.003834,MTL,Electricity Consumption,Without
2,0.023504,-0.024056,MTL,Electricity Consumption,Without
3,0.018734,-0.011009,MTL,Electricity Consumption,Without
4,0.016716,0.021324,MTL,Electricity Consumption,Without
...,...,...,...,...,...
9067,0.314016,0.300900,MTL,Temperature Closer to Evaporator (C),Hybrid - Filter (SelectKBest) + Embedded (RF)
9068,0.212938,0.205123,MTL,Temperature Closer to Evaporator (C),Hybrid - Filter (SelectKBest) + Embedded (RF)
9069,0.168464,0.194350,MTL,Temperature Closer to Evaporator (C),Hybrid - Filter (SelectKBest) + Embedded (RF)
9070,0.168464,0.187659,MTL,Temperature Closer to Evaporator (C),Hybrid - Filter (SelectKBest) + Embedded (RF)


In [3]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Calculate MAE and RMSE for each combination of Feature_Selection method and Output
grouped = final_df.groupby(['Feature_Selection', 'Output'])
metrics = []

for (method, output), group in grouped:
    true_values = group['True']
    pred_values = group['Pred']
    mae = mean_absolute_error(true_values, pred_values)
    rmse = np.sqrt(mean_squared_error(true_values, pred_values))
    metrics.append({'Method': method, 'Output': output, 'MAE': mae, 'RMSE': rmse})

metric_df = pd.DataFrame(metrics). sort_values(by='MAE')
metric_df


,Method,Output,MAE,RMSE
15,Hybrid - Filter (SelectKBest) + Embedded (RF),Electricity Consumption,0.045986,0.067768
3,Embedded - Lasso,Electricity Consumption,0.056928,0.078659
9,Filter - Correlation,Electricity Consumption,0.057394,0.082009
17,Hybrid - Filter (SelectKBest) + Embedded (RF),Temperature Closer to Evaporator (C),0.058312,0.072465
5,Embedded - Lasso,Temperature Closer to Evaporator (C),0.058466,0.074259
6,Embedded - Tree Importance (Extra Trees),Electricity Consumption,0.059699,0.082814
0,Embedded - ElasticNet,Electricity Consumption,0.060150,0.079535
24,Wrapper - RFE (LinearRegression),Electricity Consumption,0.065468,0.088448
11,Filter - Correlation,Temperature Closer to Evaporator (C),0.073753,0.095206
21,Wrapper - Forward Selection,Electricity Consumption,0.075385,0.103825


In [4]:
import pandas as pd
import joblib
# Load the saved scalers
scalers_y_loaded = joblib.load('scalers_y_multi_targets.joblib')
targets = output_names

# Update the loop to filter rows by target and then apply the inverse transform
for target in targets:
    # Filter rows for the current target
    target_rows = final_df[final_df['Output'] == target]
    
    # Check if there are any rows to update to avoid KeyErrors
    if not target_rows.empty:
        # Apply inverse transformation to the true values for the current target
        true_values_transformed = scalers_y_loaded[target].inverse_transform(target_rows[['True']].values)
        pred_values_transformed = scalers_y_loaded[target].inverse_transform(target_rows[['Pred']].values)
        
        # Update the DataFrame with the inverse transformed values
        final_df.loc[final_df['Output'] == target, 'True'] = true_values_transformed
        final_df.loc[final_df['Output'] == target, 'Pred'] = pred_values_transformed

# Now final_df should have the true and predicted values back in their original scale
final_df.to_csv('final_df_FDCS_1_MTL.csv')
final_df

,True,Pred,Model,Output,Feature_Selection
0,172.61,187.797760,MTL,Electricity Consumption,Without
1,173.35,92.561447,MTL,Electricity Consumption,Without
2,188.45,21.634308,MTL,Electricity Consumption,Without
3,171.72,67.394897,MTL,Electricity Consumption,Without
4,164.64,180.803452,MTL,Electricity Consumption,Without
...,...,...,...,...,...
9067,11.83,11.732678,MTL,Temperature Closer to Evaporator (C),Hybrid - Filter (SelectKBest) + Embedded (RF)
9068,11.08,11.022009,MTL,Temperature Closer to Evaporator (C),Hybrid - Filter (SelectKBest) + Embedded (RF)
9069,10.75,10.942078,MTL,Temperature Closer to Evaporator (C),Hybrid - Filter (SelectKBest) + Embedded (RF)
9070,10.75,10.892427,MTL,Temperature Closer to Evaporator (C),Hybrid - Filter (SelectKBest) + Embedded (RF)


## GridSearch/Keras Tuner

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')


df = pd.read_csv(r'/home/lixna37/Downloads/FDCSs_Feb_2024/Datasets/FDCS_1/FDCS_1_Feb_2024.csv')
output_names = ['Electricity Consumption', 'Humidity Closer to Evaporator (%)', 'Temperature Closer to Evaporator (C)']

from MTL_FDCS_1_pre_processing import preprocess_data
X_train_normalized, y_train_normalized, X_val_normalized, y_val_normalized, X_test_normalized, y_test_normalized = preprocess_data(df)

# Combine the training and validation sets directly after they have been recalled
X_train_val_normalized = np.concatenate([X_train_normalized, X_val_normalized], axis=0)
y_train_val_normalized = np.concatenate([y_train_normalized, y_val_normalized], axis=0)

print("Shape of X_train_normalized:", X_train_normalized.shape)
print("Shape of y_train:", y_train_normalized.shape)
print("Shape of X_val_normalized:", X_val_normalized.shape)
print("Shape of y_val:", y_val_normalized.shape)
print("Shape of X_test_normalized:", X_test_normalized.shape)
print("Shape of y_test:", y_test_normalized.shape)
print("Shape of X_train_val_normalized:", X_train_val_normalized.shape)
print("Shape of y_train_val:", y_train_val_normalized.shape)

Shape of X_train_normalized: (1272, 85)
Shape of y_train: (1272, 3)
Shape of X_val_normalized: (336, 85)
Shape of y_val: (336, 3)
Shape of X_test_normalized: (336, 85)
Shape of y_test: (336, 3)
Shape of X_train_val_normalized: (1608, 85)
Shape of y_train_val: (1608, 3)


In [2]:
from FDCS_1_MLT_GridSearch import evaluate_model_with_tuner

# Call the function
best_hyperparams, metrics_df, pred_true_TestSet = evaluate_model_with_tuner(
    X_train_normalized,
    y_train_normalized,
    X_val_normalized,
    y_val_normalized,
    output_names
)


metrics_df


Trial 10 Complete [00h 00m 12s]
val_loss: 0.3243689760565758

Best val_loss So Far: 0.11876774206757545
Total elapsed time: 00h 01m 58s
Epoch 1/100
40/40 [==============================] - 1s 3ms/step - loss: 0.2826 - dense_7_loss: 0.0938 - dense_8_loss: 0.1051 - dense_9_loss: 0.0836 - dense_7_mse: 0.0938 - dense_8_mse: 0.1051 - dense_9_mse: 0.0836
Epoch 2/100
40/40 [==============================] - 0s 3ms/step - loss: 0.1955 - dense_7_loss: 0.0657 - dense_8_loss: 0.0657 - dense_9_loss: 0.0641 - dense_7_mse: 0.0657 - dense_8_mse: 0.0657 - dense_9_mse: 0.0641
Epoch 3/100
40/40 [==============================] - 0s 3ms/step - loss: 0.1865 - dense_7_loss: 0.0619 - dense_8_loss: 0.0629 - dense_9_loss: 0.0617 - dense_7_mse: 0.0619 - dense_8_mse: 0.0629 - dense_9_mse: 0.0617
Epoch 4/100
40/40 [==============================] - 0s 3ms/step - loss: 0.1828 - dense_7_loss: 0.0606 - dense_8_loss: 0.0616 - dense_9_loss: 0.0606 - dense_7_mse: 0.0606 - dense_8_mse: 0.0616 - dense_9_mse: 0.0606
Epoc

,Fold,Set,Model,Output,MAE,RMSE
0,Final,Test,MTL,Electricity Consumption,0.187332,0.243134
1,Final,Test,MTL,Humidity Closer to Evaporator (%),0.129914,0.155930
2,Final,Test,MTL,Temperature Closer to Evaporator (C),0.127582,0.145156


In [3]:
best_hyperparams

{'units_1': 10, 'units_2': 10, 'learning_rate': 0.001954220611224397}

In [4]:
pred_true_TestSet

,True,Pred,Model,Output
0,0.004761,0.418643,MTL,Electricity Consumption
1,0.002706,0.435705,MTL,Electricity Consumption
2,0.003977,0.445145,MTL,Electricity Consumption
3,0.005189,0.439359,MTL,Electricity Consumption
4,0.005006,0.413817,MTL,Electricity Consumption
...,...,...,...,...
1003,0.326146,0.354285,MTL,Temperature Closer to Evaporator (C)
1004,0.303235,0.343624,MTL,Temperature Closer to Evaporator (C)
1005,0.235849,0.362303,MTL,Temperature Closer to Evaporator (C)
1006,0.235849,0.294750,MTL,Temperature Closer to Evaporator (C)


In [5]:
pred_true_TestSet.to_csv('FDCS_1_MTL_GridSearch.csv')